## Implementación de clustering para la selección de sucursales.


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

In [31]:
amazon_delivery_drones = pd.read_csv('AmazonDroneDelivery.csv', sep='\t', index_col=0)
amazon_delivery_drones_case_2 = pd.read_csv('AmazonDroneDeliveryCase2.csv', sep='\t', index_col=0)
amazon_delivery_drones.head()

,DEMAND,latitude,longitude,NODE_TYPE
4,0.000000,4.581430,-74.096940,warehouse
5,326.792657,4.601168,-74.087721,delivery_point
6,240.372859,4.594741,-74.091919,delivery_point
7,0.000000,4.583549,-74.095351,warehouse
8,292.038180,4.595307,-74.105222,delivery_point


# Caso 1 plot global

In [32]:
fig = px.scatter_mapbox(amazon_delivery_drones,
                        lat="latitude",
                        lon="longitude",
                        color="NODE_TYPE",
                        title="Amazon Delivery Drones",
                        zoom=5,

                        )
fig.update_layout(
                    mapbox_center_lat = 4.570868,
                    mapbox_center_lon = -74.297333,
                    mapbox_style="open-street-map",
                )
fig.show()
print('plot complete')

plot complete


# Caso 2 plot global

In [33]:
fig = px.scatter_mapbox(amazon_delivery_drones_case_2,
                        lat="latitude",
                        lon="longitude",
                        color="NODE_TYPE",
                        title="Amazon Delivery Drones Case 2",
                        zoom=5,

                        )
fig.update_layout(
                    mapbox_center_lat = 4.570868,
                    mapbox_center_lon = -74.297333,
                    mapbox_style="open-street-map",
                )
fig.show()
print('plot complete')

plot complete


## Hallar la matriz de distances entre las sucursales

In [34]:
# Create a dictionary to store the distances
from haversine import haversine, Unit
distances = {}
# Iterate over the rows of the dataframe
for index, row in amazon_delivery_drones.iterrows():
    for index2, row2 in amazon_delivery_drones.iterrows():
        # Calculate the distance between the two rows using the haversine formula
        if index != index2:

           distance = haversine((row['latitude'], row['longitude']), (row2['latitude'], row2['longitude']))
        else:
            distance=999

        # Add the distance to the dictionary
        distances[(index, index2)] = distance


distances_case_2 = {}
# Iterate over the rows of the dataframe
for index, row in amazon_delivery_drones_case_2.iterrows():
    for index2, row2 in amazon_delivery_drones_case_2.iterrows():
        # Calculate the distance between the two rows using the haversine formula
        if index != index2:

           distance = haversine((row['latitude'], row['longitude']), (row2['latitude'], row2['longitude']))
        else:
            distance=999

        # Add the distance to the dictionary
        distances_case_2[(index, index2)] = distance


In [35]:
# Create a csv file with the distances
import csv
print(amazon_delivery_drones.shape)

last_index = amazon_delivery_drones.index.max()

print(last_index)
with open('distances.csv', 'w') as f:
    f.write(' \t')
    for index, row in amazon_delivery_drones.iterrows():
        node = row['NODE_TYPE']
        if  index == last_index:
            f.write( f'{node}_{index}')
        else:
            f.write( f'{node}_{index}\t')

    f.write('\n')

    for index, row in amazon_delivery_drones.iterrows():
        node = row['NODE_TYPE']
        f.write(f'{node}_{index}\t')

        for index2, row2 in amazon_delivery_drones.iterrows():
            if index2 == last_index:
                f.write(f'{np.round(distances[(index, index2)],3)}')
            else:
                f.write(f'{np.round(distances[(index, index2)],3)}\t')
        count_col = 0
        f.write('\n')
last_index_case_2 = amazon_delivery_drones_case_2.index.max()

print(last_index_case_2)
with open('distances_case_2.csv', 'w') as f:
    f.write(' \t')
    for index, row in amazon_delivery_drones_case_2.iterrows():
        node = row['NODE_TYPE']
        if  index == last_index_case_2:
            f.write( f'{node}_{index}')
        else:
            f.write( f'{node}_{index}\t')

    f.write('\n')

    for index, row in amazon_delivery_drones_case_2.iterrows():
        node = row['NODE_TYPE']
        f.write(f'{node}_{index}\t')

        for index2, row2 in amazon_delivery_drones_case_2.iterrows():
            if index2 == last_index_case_2:
                f.write(f'{np.round(distances_case_2[(index, index2)],3)}')
            else:
                f.write(f'{np.round(distances_case_2[(index, index2)],3)}\t')
        count_col = 0
        f.write('\n')

(435, 4)
622
583


In [36]:
# Create the DEMAND dictionary
demand = {}
for index, row in amazon_delivery_drones.iterrows():
    demand[index] = (f'{row["NODE_TYPE"]}_{index}',row['DEMAND'])

# Create the DEMAND csv file
with open('demand.csv', 'w') as f:
    for key in demand.keys():
        f.write(f'{demand[key][0]}\t{np.round(demand[key][1],0)}\n')

# Create the DEMAND dictionary
demand_case_2 = {}
for index, row in amazon_delivery_drones_case_2.iterrows():
    demand_case_2[index] = (f'{row["NODE_TYPE"]}_{index}',row['DEMAND'])

# Create the DEMAND csv file
with open('demand_case_2.csv', 'w') as f:
    for key in demand_case_2.keys():
        f.write(f'{demand_case_2[key][0]}\t{np.round(demand_case_2[key][1],0)}\n')

In [37]:
# Get the mean of distances
mean_distance = np.mean(list(distances.values()))
mean_distance = np.round(mean_distance, 0)
# Get the mean of demands
mean_demand = 0
for index, row in amazon_delivery_drones.iterrows():
    if row['NODE_TYPE'] == 'delivery_point':
        mean_demand += row['DEMAND']

mean_demand = mean_demand/len(amazon_delivery_drones[amazon_delivery_drones['NODE_TYPE'] == 'delivery_point'])


mean_demand = np.round(mean_demand,0)

print(f'Distancia media: {mean_distance}[km]')
print(f'Demanda media: {mean_demand}[kg]')

# Get the mean of distances
mean_distance_case_2 = np.mean(list(distances_case_2.values()))
mean_distance_case_2 = np.round(mean_distance_case_2, 0)
# Get the mean of demands
mean_demand_case_2 = 0
for index, row in amazon_delivery_drones_case_2.iterrows():
    if row['NODE_TYPE'] == 'delivery_point':
        mean_demand_case_2 += row['DEMAND']

mean_demand_case_2 = mean_demand_case_2/len(amazon_delivery_drones_case_2[amazon_delivery_drones_case_2['NODE_TYPE'] == 'delivery_point'])


mean_demand_case_2 = np.round(mean_demand_case_2,0)

print(f'Distancia media: {mean_distance_case_2}[km]')
print(f'Demanda media: {mean_demand_case_2}[kg]')

Distancia media: 13.0[km]
Demanda media: 238.0[kg]
Distancia media: 60.0[km]
Demanda media: 282.0[kg]


In [38]:
# Create a random uniform capacity for the drones
n_drones = len(amazon_delivery_drones[amazon_delivery_drones['NODE_TYPE'] == 'warehouse']) * 3
capacity = np.random.uniform(0.5*mean_demand, 1.5*mean_demand, n_drones)
capacity = np.round(capacity,0)
# Create a random uniform battery range for the drones
mean_battery_range = 4*mean_distance
battery_range = np.random.uniform(0.5*mean_battery_range, 1.5*mean_battery_range, n_drones)

# Create a random uniform capacity for the drones
n_drones_case_2 = len(amazon_delivery_drones_case_2[amazon_delivery_drones_case_2['NODE_TYPE'] == 'warehouse']) * 3
capacity_case_2 = np.random.uniform(0.5*mean_demand_case_2, 1.5*mean_demand_case_2, n_drones_case_2)
capacity_case_2 = np.round(capacity_case_2,0)
# Create a random uniform battery range for the drones
mean_battery_range_case_2 = 4*mean_distance_case_2
battery_range_case_2 = np.random.uniform(0.5*mean_battery_range_case_2, 1.5*mean_battery_range_case_2, n_drones_case_2)

In [39]:
# Create the info_drones dictionary
info_drone_dict = {}
for i in range(n_drones):
    info_drone_dict[i] = (f'drone_{i}', capacity[i], battery_range[i])

# Create the info_drones.csv file
with open('info_drones.csv', 'w') as f:
    for key in info_drone_dict.keys():
        f.write(f'{info_drone_dict[key][0]}\t{np.round(info_drone_dict[key][1],0)}\t{np.round(info_drone_dict[key][2],0)}\n')

# Create the info_drones dictionary
info_drone_dict_case_2 = {}
for i in range(n_drones_case_2):
    info_drone_dict_case_2[i] = (f'drone_{i}', capacity_case_2[i], battery_range_case_2[i])

# Create the info_drones.csv file
with open('info_drones_case_2.csv', 'w') as f:
    for key in info_drone_dict_case_2.keys():
        f.write(f'{info_drone_dict_case_2[key][0]}\t{np.round(info_drone_dict_case_2[key][1],0)}\t{np.round(info_drone_dict_case_2[key][2],0)}\n')

# One hot encoding NODE_TYPE

In [40]:
# Make one hot encoding for the NODE_TYPE column
amazon_delivery_drones = pd.get_dummies(amazon_delivery_drones, columns=['NODE_TYPE'])
amazon_delivery_drones_case_2 = pd.get_dummies(amazon_delivery_drones_case_2, columns=['NODE_TYPE'])

# Create an array that contains wich index is a warehouse

warehouse_index = amazon_delivery_drones[amazon_delivery_drones['NODE_TYPE_warehouse'] == 1].index

warehouse_index_case_2 = amazon_delivery_drones_case_2[amazon_delivery_drones_case_2['NODE_TYPE_warehouse'] == 1].index

# Create an array that constains wich index is a delivery point

delivery_point_index = amazon_delivery_drones[amazon_delivery_drones['NODE_TYPE_delivery_point'] == 1].index

delivery_point_index_case_2 = amazon_delivery_drones_case_2[amazon_delivery_drones_case_2['NODE_TYPE_delivery_point'] == 1].index

# Create an array that contains wich index is a charging station

charging_station_index = amazon_delivery_drones[amazon_delivery_drones['NODE_TYPE_charging_station'] == 1].index

charging_station_index_case_2 = amazon_delivery_drones_case_2[amazon_delivery_drones_case_2['NODE_TYPE_charging_station'] == 1].index